<a href="https://colab.research.google.com/github/NotAndex/Demo/blob/main/time_series_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0

# 1 Setup


In [ ]:
!pip install yfinance
import yfinance as yf

import numpy as np
import pandas as pd
import datetime

# 2 Data

In [4]:
today = datetime.date.today()
start_date = today - datetime.timedelta(days=365*3)
symbol = 'BTC-USD'

cc_data = yf.download(symbol, start=start_date, end=today)
cc_data.reset_index(inplace=True,col_fill=('Date'))

[*********************100%***********************]  1 of 1 completed


#3 Feature engineering examples

## 3.1 Date dependent features

# 3.2 Technical features (Indicators **$f_{x}$**)

# 3.3 External Features
e.g Sentinment